In [11]:
import pandas as pd
import xgboost as xgb
import pickle

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

In [12]:
data = pd.read_csv("data_transformed.csv")

In [13]:


# Dividir los datos en características (X) y variable objetivo (y)
X = data.drop('readmitted', axis=1)
y = data['readmitted']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verificar las dimensiones de los conjuntos de datos
X_train.shape, X_test.shape

((81412, 29), (20354, 29))

In [14]:
# Separar el conjunto de entrenamiento por clases
X_train_majority = X_train[y_train == 0]
X_train_minority = X_train[y_train == 1]
y_train_majority = y_train[y_train == 0]
y_train_minority = y_train[y_train == 1]

# Realizar el sobremuestreo
X_train_minority_oversampled = X_train_minority.sample(len(X_train_majority), replace=True, random_state=42)
y_train_minority_oversampled = y_train_minority.sample(len(y_train_majority), replace=True, random_state=42)

# Combinar las clases mayoritarias y minoritarias sobremuestreadas
X_train_oversampled = pd.concat([X_train_majority, X_train_minority_oversampled])
y_train_oversampled = pd.concat([y_train_majority, y_train_minority_oversampled])

# Verificar el equilibrio de las clases después del sobremuestreo
y_train_oversampled.value_counts()


readmitted
0    72326
1    72326
Name: count, dtype: int64

In [15]:
# Imputar valores faltantes con la mediana
median_value = X_train['time_in_hospital'].median()
X_train['time_in_hospital'].fillna(median_value, inplace=True)
X_test['time_in_hospital'].fillna(median_value, inplace=True)

# Verificar si aún hay valores faltantes
missing_values_after = X_train.isnull().sum().sum()
missing_values_after


0

In [16]:

# Guardar la mediana en un archivo pickle
pickle_filename = "median_time_in_hospital.pkl"
with open(pickle_filename, 'wb') as file:
    pickle.dump(median_value, file)

pickle_filename


'median_time_in_hospital.pkl'

In [17]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=10000),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "XGBoost": xgb.XGBClassifier()
}

for name, model in models.items():
    print(f"Evaluating model: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("-" * 60)

Evaluating model: Logistic Regression


/Users/anestrada/opt/anaconda3/envs/Prueba_tecnica_MeLi_2023/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anestrada/opt/anaconda3/envs/Prueba_tecnica_MeLi_2023/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anestrada/opt/anaconda3/envs/Prueba_tecnica_MeLi_2023/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18083
           1       0.00      0.00      0.00      2271

    accuracy                           0.89     20354
   macro avg       0.44      0.50      0.47     20354
weighted avg       0.79      0.89      0.84     20354

[[18083     0]
 [ 2271     0]]
------------------------------------------------------------
Evaluating model: Random Forest
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18083
           1       0.60      0.02      0.03      2271

    accuracy                           0.89     20354
   macro avg       0.74      0.51      0.49     20354
weighted avg       0.86      0.89      0.84     20354

[[18059    24]
 [ 2235    36]]
------------------------------------------------------------
Evaluating model: SVM


/Users/anestrada/opt/anaconda3/envs/Prueba_tecnica_MeLi_2023/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anestrada/opt/anaconda3/envs/Prueba_tecnica_MeLi_2023/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anestrada/opt/anaconda3/envs/Prueba_tecnica_MeLi_2023/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18083
           1       0.00      0.00      0.00      2271

    accuracy                           0.89     20354
   macro avg       0.44      0.50      0.47     20354
weighted avg       0.79      0.89      0.84     20354

[[18083     0]
 [ 2271     0]]
------------------------------------------------------------
Evaluating model: XGBoost
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     18083
           1       0.49      0.04      0.07      2271

    accuracy                           0.89     20354
   macro avg       0.69      0.52      0.51     20354
weighted avg       0.85      0.89      0.84     20354

[[17989    94]
 [ 2182    89]]
------------------------------------------------------------


In [18]:


# Definición de la grilla de hiperparámetros para XGBoost
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'min_child_weight': [1, 5, 10]
}

models = {
    "XGBoost": xgb.XGBClassifier()
}

for name, model in models.items():
    print(f"Evaluating model: {name}")
    
    if name == "XGBoost":  # Si el modelo es XGBoost, hacemos el tuning
        search = RandomizedSearchCV(model, param_distributions=xgb_param_grid, n_iter=50, scoring='f1', n_jobs=-1, cv=5, verbose=3)
        search.fit(X_train, y_train)
        best_model = search.best_estimator_  # Usamos el mejor modelo encontrado
    else:
        best_model = model
        best_model.fit(X_train, y_train)
        
    y_pred = best_model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("-" * 60)


Evaluating model: XGBoost
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END colsample_bytree=0.9, gamma=0.4, learning_rate=0.2, max_depth=7, min_child_weight=1, n_estimators=50, subsample=1.0;, score=0.055 total time=   2.0s
[CV 2/5] END colsample_bytree=0.9, gamma=0.4, learning_rate=0.2, max_depth=7, min_child_weight=1, n_estimators=50, subsample=1.0;, score=0.051 total time=   2.1s
[CV 3/5] END colsample_bytree=0.9, gamma=0.4, learning_rate=0.2, max_depth=7, min_child_weight=1, n_estimators=50, subsample=1.0;, score=0.053 total time=   2.0s
[CV 4/5] END colsample_bytree=0.9, gamma=0.4, learning_rate=0.2, max_depth=7, min_child_weight=1, n_estimators=50, subsample=1.0;, score=0.056 total time=   2.0s
[CV 3/5] END colsample_bytree=0.9, gamma=0.4, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=200, subsample=0.9;, score=0.041 total time=   5.9s
[CV 2/5] END colsample_bytree=0.9, gamma=0.4, learning_rate=0.05, max_depth=7, min_child_weight=5, n

In [ ]:
# Sample data (replace with your data)
X_train, y_train = X_train, y_train

# Initialize and train the XGBoost model with the best hyperparameters
model = xgb.XGBClassifier(
    colsample_bytree=0.7, 
    gamma=0.2, 
    learning_rate=0.05, 
    max_depth=3, 
    min_child_weight=5, 
    n_estimators=100, 
    subsample=1.0
)
model.fit(X_train, y_train)

# Save the trained model to a file
joblib.dump(model, "best_xgboost_model.pkl")
